# Project 1, Part 4, Holiday Analytics

University of California, Berkeley
Master of Information and Data Science (MIDS) program
w205 - Fundamentals of Data Engineering

Student: John (Jack) Galvin

Year: 2022

Semester: Spring

Section: 9


# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
# Function to run a select query and return rows in a pandas dataframe
# Pandas puts all numeric values from postgres to float
# If it will fit in an integer, change it to integer


def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
# Connect to Postgres

connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
# Create a cursor for the connection

cursor = connection.cursor()

## How do holidays affect the sales, considering both the actual holiday, and the days before and after the holiday? 

## Create an executive summary explaining how holidays have affected sales. 

## You must support your summary with data, in the form of output of queries, data visualization, etc. There is a 1 query minimum.

In [82]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select sale_date,
        total_amount
from sales
order by sale_date

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,sale_date,total_amount
0,2020-01-01,96
1,2020-01-01,144
2,2020-01-01,48
3,2020-01-01,24
4,2020-01-01,108
...,...,...
1537612,2020-12-31,60
1537613,2020-12-31,96
1537614,2020-12-31,60
1537615,2020-12-31,12


In [75]:
df.total_amount.sum()

98739408

In [ ]:
# produce a df with just date and usd (like above). use pandas to roll up to daily.
# produce a separate df with just date and total quantity sold
# join df 1 and 2 together

# Then, merge in holidays df

In [77]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select sa.sale_date as date,
        sum(l.quantity)
from sales as sa
    join line_items as l
        on sa.store_id = l.store_id and sa.sale_id = l.sale_id
group by date
order by date

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,date,sum
0,2020-01-01,11148
1,2020-01-02,21760
2,2020-01-03,21923
3,2020-01-04,33364
4,2020-01-05,32916
...,...,...
359,2020-12-27,33008
360,2020-12-28,22071
361,2020-12-29,10870
362,2020-12-30,21952
